# 🌌 Quantum Foam Stability Simulation with Emanator Modulation

This notebook simulates quantum foam (spacetime fluctuations) in a two-qubit system, testing whether modulation by the golden ratio φ (Phi ≈ 1.618) can stabilize entanglement and reduce variance in coherence measures. The simulation leverages the QuTiP library to model random perturbations and decoherence, reflecting Planck-scale spacetime 'bubbling'.

**Hypothesis:** Emanator-inspired φ modulation reduces entanglement variance and preserves coherence, mirroring the sacred geometry's hypothesized stabilizing influence.

In [ ]:
# Imports and initialization
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt

# Helper function for von Neumann entropy
def von_neumann_entropy(rho):
    evals = np.linalg.eigvalsh(rho.full())
    evals = evals[evals > 1e-10]
    return -np.sum(evals * np.log(evals))

# Parameters setup
N = 2  # Two qubits
gamma = 0.1
omega = 1.0
phi = (1 + np.sqrt(5)) / 2
tlist = np.linspace(0, 10, 100)
seed = 42
np.random.seed(seed)

## ⚙️ Defining Hamiltonians with Quantum Foam

Define two Hamiltonians:
- **Base Hamiltonian:** Standard with random noise (quantum foam).
- **Phi-Modulated Hamiltonian:** Frequencies scaled by φ and foam attenuated by 1/φ.

In [ ]:
# Base Hamiltonian
H_base = omega * (qt.tensor(qt.sigmaz(), qt.qeye(2)) + qt.tensor(qt.qeye(2), qt.sigmaz())) / 2

# Foam perturbations (random small noise)
foam_ops = [
    np.random.randn() * 0.01 * qt.tensor(qt.sigmax(), qt.qeye(2)),
    np.random.randn() * 0.01 * qt.tensor(qt.qeye(2), qt.sigmax()),
    np.random.randn() * 0.01 * qt.tensor(qt.sigmay(), qt.qeye(2)),
    np.random.randn() * 0.01 * qt.tensor(qt.qeye(2), qt.sigmay())
]
H_foam_base = H_base + sum(foam_ops)

# Decoherence operators
c_ops_base = [
    np.sqrt(gamma) * qt.tensor(qt.sigmam(), qt.qeye(2)),
    np.sqrt(gamma) * qt.tensor(qt.qeye(2), qt.sigmam())
]

# Phi-Modulated Hamiltonian
H_phi = omega * phi * (qt.tensor(qt.sigmaz(), qt.qeye(2)) + qt.tensor(qt.qeye(2), qt.sigmaz())) / 2
foam_ops_phi = [f * (1/phi) for f in foam_ops]
H_foam_phi = H_phi + sum(foam_ops_phi)

## 🌀 Running Simulations

Solve dynamics with both base and φ-modulated Hamiltonians.

In [ ]:
# Initial Bell state
psi0 = (qt.tensor(qt.basis(2,0), qt.basis(2,0)) + qt.tensor(qt.basis(2,1), qt.basis(2,1))).unit()

# Solve dynamics
result_base = qt.mesolve(H_foam_base, psi0, tlist, c_ops_base)
result_phi = qt.mesolve(H_foam_phi, psi0, tlist, c_ops_base)

# Compute entropy and concurrence
ent_base = [von_neumann_entropy(state) for state in result_base.states]
conc_base = [qt.concurrence(state) for state in result_base.states]
ent_phi = [von_neumann_entropy(state) for state in result_phi.states]
conc_phi = [qt.concurrence(state) for state in result_phi.states]

## 📈 Results and Visualization

Compare entropy and concurrence variance to assess stabilization by φ modulation.

In [ ]:
# Plotting results
plt.figure(figsize=(12,5))

# Entropy
plt.subplot(1,2,1)
plt.plot(tlist, ent_base, label='Base Entropy')
plt.plot(tlist, ent_phi, label='Phi Entropy', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Entropy')
plt.title('Entropy over Time')
plt.legend()

# Concurrence
plt.subplot(1,2,2)
plt.plot(tlist, conc_base, label='Base Concurrence')
plt.plot(tlist, conc_phi, label='Phi Concurrence', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Concurrence')
plt.title('Concurrence over Time')
plt.legend()

plt.tight_layout()
plt.show()